# Data Processing

The .ipynb file for processing the data in HSBC_Set01 folder

***Fnmatch:*** https://linuxhint.com/fnmatch-module-python/

In [1]:
import pandas as pd
import numpy as np
import os
import re
import fnmatch #to find the files which match a pattern

***Reference:*** 

https://www.tutorialspoint.com/python-read-all-csv-files-in-a-folder-in-pandas

https://www.w3schools.com/python/ref_func_zip.asp



In [2]:
# Convert a list to dictionary
def Convert(a):
    it = iter(a) # returns an iterator for the given list.
    res_dct = dict(zip(it, it)) 
    return res_dct

# Based on the value of 'bid' and 'ask', 
# assign the corresponding value to the column 'State' 
def check_state(bid,ask):
    if bid==1 and ask==0:
        return 'bid'
    elif bid==0 and ask==1:
        return 'ask'
    elif bid==1 and ask==1:
        return 'bid/ask'
    else:
        return 'Nan'


# Pass the merged dataframe and the origin LOB dataframe to the function
# For each time point, match a corresponding state(bid or ask) with it
# 'result' is the Tapes dataframe, 'df' is the LOBs dataframe
def Match_bid_ask(result,df):
    # get every row of the dataframe, get the index and each data point
    for index, rows in result.iterrows():
        bid_in=0
        ask_in=0

        # make the Price and Qty be paired together
        match_array=np.array([rows.Price,rows.Qty])
        bid_array=np.array(rows.bid)
        ask_array=np.array(rows.ask)
        for i in bid_array:
            # check whether the matched array is completely equal to any array in the bid feature.
            if np.array_equal(match_array,i):
                bid_in=1
                break
            # check whether the bid array includes the same price which corresponds to the Price
            elif rows.Price==i[0]:
                # get the index of corresponding "Time" feature in LOBs dataframe
                index_df=df[df['Time']==rows.Time].index.values

                # Scan the next line of the current line in LOBs dataframe
                # get the data of "bid" feature
                df_bid=df.loc[index_df[0]+1]["bid"]

                # check whether the bid array includes the same price 
                # which corresponds to the Price in the current row
                if np.any(np.isin(df_bid,rows.Price)):
                    # scan each data point in the bid array
                    for k in df_bid:
                        # Under the same price, check whether the quantity decreases
                        if k[0]==i[0] and k[1]<i[1]:
                            # if it decreases at the next time point,
                            # means that there is a deal at the current time
                            # let the value of "bid_in" be 1, means the type of deal is "bid"
                            bid_in=1
                            break
                else:
                    bid_in=1
                    break
        # Do the same thing to "ask" feature as the "bid" feature above
        for j in ask_array:
            if np.array_equal(match_array,j):
                ask_in=1
                break
            elif rows.Price==j[0]:
                index_df=df[df['Time']==rows.Time].index.values
                df_ask=df.loc[index_df[0]+1]["ask"]
                if np.any(np.isin(df_ask,rows.Price)):
                    for k in df_ask:
                        if k[0]==j[0] and k[1]<j[1]:
                            ask_in=1
                            break
                else:
                    ask_in=1
                    break
    # Assign the type of deal("bid" or "ask") to the column of "State"
    result.at[index,'State']=check_state(bid_in,ask_in)
    return result

In [3]:
# Pass the LOB data(.txt file) and Tapes data(.csv file) to the function
# Tidy up the data and find the corresponding state for each time point in Tapes data
def data_wrangling(LOBs_data,Tapes_data):
    # list to array
    New_data=[]
    test_lines=np.asarray(LOBs_data,dtype=object)

    # Normalize each row of LOBs data to join with Tapes data
    # The format should be [["Time", 5.673],["Encode", "Exch0"],["bid",[]],["ask",[]]]
    for i in test_lines:
        every_line=[["Time",i[0]],["Encode",i[1]],i[2][0],i[2][1]]
        New_data.append(every_line)

    # Convert every list in the data to dictionary
    # E.g. ["Time", 5.673] to {"Time": 5.673}
    for i in New_data:
        i[0]=Convert(i[0])
        i[1]=Convert(i[1])
        i[2]=Convert(i[2])
        i[3]=Convert(i[3])
        # Merge the dictionaries
        # {"Time":5.673, "Encode": "Exch0", "bid": [], "ask": []}
        i[0]=i[0]|i[1]|i[2]|i[3] 

    # delete the redundant keys and values
    New_data=np.delete(New_data,[1,2,3],1)
    data_test=New_data.flatten()
    #Convert the array to list
    data_test2=data_test.tolist()

    # Convert the precessed LOBs data to dataframe
    df=pd.DataFrame(data_test2)

    # join the Tapes data with LoBs data in the form of "inner".
    # The common key is "Time"
    merge_table = pd.merge(Tapes_data,df, on='Time', how="inner")

    # Based on the features "Time","bid","ask", 
    # to find the State for each time point 
    result=Match_bid_ask(merge_table,df)
    return result

In [7]:
columns_name=['Time','Price','Qty']
# Read all .csv file in Tapes folder
# get the name of .csv file

folder_path = "./Dataset/HSBC_Set01/Tapes/"
new_folder_path="./Processed_data/"
# define the prefix for the new file names
prefix = "New_"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # read the csv file
        Tapedata = pd.read_csv(os.path.join(folder_path, filename),names=columns_name,header=None)
        
        # add a feature/column called "State"
        Tapedata["State"]=" "

        # get the name of filename excluding the suffix .csv
        name=os.path.splitext(filename)[0]

        # a pattern to get the date embedded in the filename
        pattern = r"\d{4}-\d{2}-\d{2}"
        match=re.search(pattern,name)
        if match:
            # print the matched date string
            Date_match=match.group(0)
            # read the single file
            LOBs_folder_path=r'C:\Users\hp\Documents\DS_miniproject\HSBC_Set01\LOBS'
            LOB_file=""

            # Based on the date, 
            # find .txt file whose filename has the same date as .csv file
            for file in os.listdir(LOBs_folder_path):
                if fnmatch.fnmatch(file, f'*{Date_match}*') and file.endswith(".txt"):
                    LOB_file=file
                    break

            # Convert Exch0 to the type "String"
            substring="Exch0"
            LOBdata=[]
            with open(os.path.join(LOBs_folder_path, LOB_file),'r') as f:
                for line in f:
                    # add quotation marks around the substring using an f-string
                    new_line = line.replace(substring, f'"{substring}"')

                    # remove the quotation marks around the line
                    # Convert the string to the list
                    LOBdata.append(eval(new_line.strip('\n')))
            
            # Process the data
            Result=data_wrangling(LOBdata,Tapedata)

            # write the new data to the new file
            # save the file to 'Processed_data' folder
            new_filename=prefix+name+".csv"
            Result.to_csv(os.path.join(new_folder_path, new_filename), index=False)

        else:
            print("No match found.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Obtain stock data
df = pd.read_csv('./Dataset/HSBC_Set01/Tapes/.csv')

# Step 2: Load data into a pandas DataFrame

# Step 3: Convert the date column to a datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Step 4: Set the date column as the index
df.set_index('Date', inplace=True)

# Step 5: Create a time series plot
df['Close'].plot()
plt.title('Stock Status')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

#Data visualization

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(BID[:, 0], BID[:, 1], c='green', alpha=0.6, edgecolors='black', label='BID', s=60)
plt.scatter(ASK[:, 0], ASK[:, 1], c='red', alpha=0.6, edgecolors='black', label='ASK', s=60)
plt.legend(loc='lower left')

In [ ]:
plt.subplot(221)
plt.plot(df.Close, '-', label='By Days')
plt.legend()
plt.show()

# Stock market prices are highly unpredictable and volatile. This means that there are no consistent patterns in the data that allwows us to model stock prices over time perfectly. However, we can model the data, so that the predictions we make can somehow correlate with the actual behavior of the data. We try to do that with Long-Term Short Memory (LSTM) models. 

In [ ]:
#importing necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler